## Linking a dataset of real historical persons

In this example, we deduplicate a more realistic dataset. The data is based on historical persons scraped from wikidata. Duplicate records are introduced with a variety of errors introduced.


<a target="_blank" href="https://colab.research.google.com/github/moj-analytical-services/splink/blob/splink4_examples_notebooks/docs/demos/examples/duckdb/deduplicate_50k_synthetic.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [1]:
# Uncomment and run this cell if you're running in Google Colab.
# !pip install git+https://github.com/moj-analytical-services/splink.git@splink4_examples_notebooks

In [2]:
from splink.datasets import splink_datasets


df = splink_datasets.historical_50k
df.head()

,unique_id,cluster,full_name,first_and_surname,first_name,surname,dob,birth_place,postcode_fake,gender,occupation
0,Q2296770-1,Q2296770,"thomas clifford, 1st baron clifford of chudleigh",thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8df,male,politician
1,Q2296770-2,Q2296770,thomas of chudleigh,thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8df,male,politician
2,Q2296770-3,Q2296770,tom 1st baron clifford of chudleigh,tom chudleigh,tom,chudleigh,1630-08-01,devon,tq13 8df,male,politician
3,Q2296770-4,Q2296770,thomas 1st chudleigh,thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8hu,None,politician
4,Q2296770-5,Q2296770,"thomas clifford, 1st baron chudleigh",thomas chudleigh,thomas,chudleigh,1630-08-01,devon,tq13 8df,None,politician


In [3]:
from splink.duckdb.database_api import DuckDBAPI
from splink.profile_data import profile_columns
from splink.column_expression import ColumnExpression

db_api = DuckDBAPI()
profile_columns(df, db_api, column_expressions=["first_name", "substr(surname,1,2)"])

alt.VConcatChart(...)

In [4]:
from splink.blocking_rule_library import block_on
from splink.settings_creator import SettingsCreator
from splink.linker import Linker

# Simple settings will be used for exploratory analysis


settings = SettingsCreator(
    link_type="dedupe_only",
    blocking_rules_to_generate_predictions=[
        block_on("first_name", "surname"),
        block_on("surname", "dob"),
        block_on("first_name", "dob"),
        block_on("postcode_fake", "first_name"),
    ],
)

linker = Linker(df, settings, database_api=db_api)
linker.cumulative_num_comparisons_from_blocking_rules_chart()

alt.Chart(...)

In [5]:
from splink.settings_creator import SettingsCreator
import splink.comparison_library as cl
import splink.comparison_template_library as ctl

settings = SettingsCreator(
    link_type="dedupe_only",
    blocking_rules_to_generate_predictions=[
        block_on("first_name", "surname"),
        block_on("surname", "dob"),
        block_on("first_name", "dob"),
        block_on("postcode_fake", "first_name"),
    ],
    comparisons=[
        ctl.NameComparison("first_name").configure(term_frequency_adjustments=True),
        ctl.NameComparison("surname").configure(term_frequency_adjustments=True),
        ctl.DateComparison(
            "dob",
            input_is_string=True,
            datetime_metrics=["month", "year", "year"],
            datetime_thresholds=[1, 1, 10],
        ),
        # TODO: Restore ctl.PostcodeComparison level here
        cl.LevenshteinAtThresholds("postcode_fake"),
        cl.ExactMatch("birth_place").configure(term_frequency_adjustments=True),
        cl.ExactMatch("occupation").configure(term_frequency_adjustments=True),
    ],
    retain_intermediate_calculation_columns=True,
)

linker = Linker(df, settings, database_api=db_api)

In [6]:
linker.estimate_probability_two_random_records_match(
    [
        "l.first_name = r.first_name and l.surname = r.surname and l.dob = r.dob",
        "substr(l.first_name,1,2) = substr(r.first_name,1,2) and l.surname = r.surname and substr(l.postcode_fake,1,2) = substr(r.postcode_fake,1,2)",
        "l.dob = r.dob and l.postcode_fake = r.postcode_fake",
    ],
    recall=0.6,
)

Probability two random records match is estimated to be  0.000136.
This means that amongst all possible pairwise record comparisons, one in 7,362.31 are expected to match.  With 1,279,041,753 total possible comparisons, we expect a total of around 173,728.33 matching pairs


In [7]:
linker.estimate_u_using_random_sampling(max_pairs=5e6)

----- Estimating u probabilities using random sampling -----


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).
    - dob (no m values are trained).
    - postcode_fake (no m values are trained).
    - birth_place (no m values are trained).
    - occupation (no m values are trained).


In [8]:
training_blocking_rule = block_on("first_name", "surname")
training_session_names = linker.estimate_parameters_using_expectation_maximisation(
    training_blocking_rule
)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
(l."first_name" = r."first_name") AND (l."surname" = r."surname")

Parameter estimates will be made for the following comparison(s):
    - dob
    - postcode_fake
    - birth_place
    - occupation

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - first_name
    - surname


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


Iteration 1: Largest change in params was -0.528 in probability_two_random_records_match
Iteration 2: Largest change in params was -0.0311 in probability_two_random_records_match
Iteration 3: Largest change in params was -0.0118 in the m_probability of birth_place, level `All other comparisons`
Iteration 4: Largest change in params was -0.00483 in the m_probability of birth_place, level `All other comparisons`
Iteration 5: Largest change in params was 0.00213 in the m_probability of birth_place, level `Exact match on birth_place`
Iteration 6: Largest change in params was 0.000981 in the m_probability of birth_place, level `Exact match on birth_place`
Iteration 7: Largest change in params was -0.000483 in the m_probability of dob, level `Abs difference of 'transformed dob <= 10 year'`
Iteration 8: Largest change in params was -0.000253 in the m_probability of dob, level `Abs difference of 'transformed dob <= 10 year'`
Iteration 9: Largest change in params was -0.000132 in the m_probabi

In [9]:
training_blocking_rule = block_on("dob")
training_session_dob = linker.estimate_parameters_using_expectation_maximisation(
    training_blocking_rule
)


----- Starting EM training session -----



Estimating the m probabilities of the model by blocking on:
l."dob" = r."dob"

Parameter estimates will be made for the following comparison(s):
    - first_name
    - surname
    - postcode_fake
    - birth_place
    - occupation

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - dob


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


Iteration 1: Largest change in params was -0.362 in the m_probability of first_name, level `Exact match on first_name`
Iteration 2: Largest change in params was 0.0343 in the m_probability of first_name, level `All other comparisons`
Iteration 3: Largest change in params was 0.0049 in the m_probability of first_name, level `All other comparisons`
Iteration 4: Largest change in params was 0.00109 in the m_probability of surname, level `All other comparisons`
Iteration 5: Largest change in params was 0.000261 in the m_probability of surname, level `All other comparisons`
Iteration 6: Largest change in params was 6.1e-05 in the m_probability of surname, level `All other comparisons`

EM converged after 6 iterations

Your model is fully trained. All comparisons have at least one estimate for their m and u values


The final match weights can be viewed in the match weights chart:


In [10]:
linker.match_weights_chart()

alt.VConcatChart(...)

In [11]:
linker.unlinkables_chart()

alt.LayerChart(...)

In [12]:
df_predict = linker.predict()
df_e = df_predict.as_pandas_dataframe(limit=5)
df_e

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,match_weight,match_probability,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,tf_first_name_l,tf_first_name_r,bf_first_name,...,bf_birth_place,bf_tf_adj_birth_place,occupation_l,occupation_r,gamma_occupation,tf_occupation_l,tf_occupation_r,bf_occupation,bf_tf_adj_occupation,match_key
0,-15.797376,0.000018,Q7528564-9,Q75867928-1,sir,sir,3,0.024985,0.024985,42.913167,...,0.15672,1.0,historian,military officer,0,0.012456,0.010756,0.105227,1.0,0
1,-15.797376,0.000018,Q7528564-9,Q75867928-2,sir,sir,3,0.024985,0.024985,42.913167,...,0.15672,1.0,historian,military officer,0,0.012456,0.010756,0.105227,1.0,0
2,-15.797376,0.000018,Q7528564-9,Q75867928-3,sir,sir,3,0.024985,0.024985,42.913167,...,0.15672,1.0,historian,military officer,0,0.012456,0.010756,0.105227,1.0,0
3,-15.797376,0.000018,Q7528564-9,Q75867928-4,sir,sir,3,0.024985,0.024985,42.913167,...,0.15672,1.0,historian,military officer,0,0.012456,0.010756,0.105227,1.0,0
4,-15.797376,0.000018,Q7528564-9,Q75867928-6,sir,sir,3,0.024985,0.024985,42.913167,...,0.15672,1.0,historian,military officer,0,0.012456,0.010756,0.105227,1.0,0


You can also view rows in this dataset as a waterfall chart as follows:


In [13]:
from splink.charts import waterfall_chart

records_to_plot = df_e.to_dict(orient="records")
linker.waterfall_chart(records_to_plot, filter_nulls=False)

alt.LayerChart(...)

In [14]:
clusters = linker.cluster_pairwise_predictions_at_threshold(
    df_predict, threshold_match_probability=0.95
)

Completed iteration 1, root rows count 624
Completed iteration 2, root rows count 94
Completed iteration 3, root rows count 19
Completed iteration 4, root rows count 4
Completed iteration 5, root rows count 0


In [15]:
linker.cluster_studio_dashboard(
    df_predict,
    clusters,
    "dashboards/50k_cluster.html",
    sampling_method="by_cluster_size",
    overwrite=True,
)

from IPython.display import IFrame

IFrame(src="./dashboards/50k_cluster.html", width="100%", height=1200)

In [16]:
linker.roc_chart_from_labels_column("cluster", match_weight_round_to_nearest=0.02)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

alt.Chart(...)

In [17]:
records = linker.prediction_errors_from_labels_column(
    "cluster",
    threshold=0.999,
    include_false_negatives=False,
    include_false_positives=True,
).as_record_dict()
linker.waterfall_chart(records)

alt.LayerChart(...)

In [18]:
# Some of the false negatives will be because they weren't detected by the blocking rules
records = linker.prediction_errors_from_labels_column(
    "cluster",
    threshold=0.5,
    include_false_negatives=True,
    include_false_positives=False,
).as_record_dict(limit=50)

linker.waterfall_chart(records)

alt.LayerChart(...)